In [ ]:
import os
from getpass import getpass
from google import genai

# pass your API key here
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY") or getpass(
    "Enter Google API Key: "
)
# initialize our client
client = genai.Client()

In [2]:
from IPython.display import Markdown

model_id = "gemini-2.0-flash-exp"

response = client.models.generate_content(
    model=model_id, contents='What is Gemini?'
)
Markdown(response.text)

Gemini refers to a few different things, but most commonly it refers to **Google's large language model.** Here's a breakdown:

**1. Gemini (The Google AI Model):**

*   **What it is:** Gemini is a multimodal AI model developed by Google AI. It's designed to be Google's most capable and general AI model, surpassing previous models like LaMDA and PaLM.
*   **Capabilities:** It can understand and generate text, images, audio, video, and code. This "multimodal" nature means it can process and understand information from various sources simultaneously. It's designed to be highly adaptable and perform well on a wide range of tasks.
*   **Versions:** There are different versions of Gemini optimized for various use cases:
    *   **Gemini Ultra:** The most powerful version, designed for highly complex tasks.
    *   **Gemini Pro:** A more balanced version, suitable for a wide range of applications, including powering Google's Bard (now Gemini) chatbot.
    *   **Gemini Nano:** A lightweight version designed to run on devices like smartphones and other edge devices.
*   **Use Cases:** Gemini is being integrated into various Google products and services, including:
    *   **Search:** Improving search results and providing more comprehensive answers.
    *   **Gmail & Workspace:** Helping with email composition, summarizing documents, and generating ideas.
    *   **Bard/Gemini Chatbot:** Providing more natural and helpful conversational AI.
    *   **Coding:** Assisting developers with code generation, debugging, and understanding code.
    *   **Creative Content:** Helping with writing, image generation, and other creative tasks.

**2. Gemini (The Zodiac Sign):**

*   In astrology, Gemini is the third sign of the zodiac, associated with people born between approximately May 21 and June 20. It's an air sign often associated with communication, curiosity, and adaptability.

**3. Project Gemini (NASA):**

*   In the 1960s, Project Gemini was a NASA human spaceflight program that served as a bridge between Project Mercury (the first US human spaceflight program) and Project Apollo (the program that landed humans on the Moon). Gemini developed techniques for rendezvous, docking, and spacewalking that were crucial for the success of Apollo.

**In summary, when someone mentions "Gemini" today, they are most likely referring to the Google AI model.**  To avoid confusion, it's helpful to consider the context in which the word is used.


In [4]:
from google.genai.types import Tool, GoogleSearch

search_tool = Tool(google_search=GoogleSearch())

In [ ]:
from google.genai.types import GenerateContentConfig
model_id = "gemini-2.0-flash-exp"

config = GenerateContentConfig(
    system_instruction=(
        "You are a helpful assistant that provides up to date information "
        "to help the user in their research."
    ),
    tools=[search_tool],
    response_modalities=["TEXT"],
    temperature=0.0,  # likely default
    candidate_count=1,  # likely default

)
config
response = client.models.generate_content(
    model=model_id,
    contents="Tell me the latest news in AI",
    config=config
)
Markdown(response.text)

GenerateContentConfig(http_options=None, system_instruction='You are a helpful assistant that provides up to date information to help the user in their research.', temperature=0.0, top_p=None, top_k=None, candidate_count=1, max_output_tokens=None, stop_sequences=None, response_logprobs=None, logprobs=None, presence_penalty=None, frequency_penalty=None, seed=None, response_mime_type=None, response_schema=None, routing_config=None, model_selection_config=None, safety_settings=None, tools=[Tool(retrieval=None, google_search=GoogleSearch(), google_search_retrieval=None, enterprise_web_search=None, google_maps=None, code_execution=None, function_declarations=None)], tool_config=None, labels=None, cached_content=None, response_modalities=['TEXT'], media_resolution=None, speech_config=None, audio_timestamp=None, automatic_function_calling=None, thinking_config=None)

In [49]:
from dataclasses import Field
from langchain_tavily import TavilySearch
from langgraph.prebuilt import create_react_agent
from langchain_deepseek import ChatDeepSeek
from dotenv import load_dotenv
from typing import TypedDict, List, Annotated, Literal, Dict, Union, Optional 
from pydantic import BaseModel, Field
from langchain_core.runnables import RunnableLambda, RunnableSequence
import json

class Citation(BaseModel):
    label: str = Field(
        ...,
        description="The name of the website.",
    )
    source_id: str = Field(
        ...,
        description="The url of a SPECIFIC source which justifies the answer.",
    )
    quote: str = Field(
        ...,
        description="The VERBATIM quote from the specified source that justifies the answer.",
    )


class QuotedAnswer(BaseModel):
    """Answer the user question based only on the given sources, and return all the sources used."""
    answer: str = Field(
        ...,
        description="The answer to the user question, which is based only on the given sources. Include any relevant sources in the answer as markdown hyperlinks. For example: 'This is a sample text ([url website](url))'"
    )
    citations: List[Citation] = Field(
        ..., description="Citations from the given sources that justify the answer."
    )

load_dotenv()
llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=1,
    timeout=None,
    max_retries=2,
    )


# Initialize Tavily Search Tool
tavily_search_tool = TavilySearch(
    max_results=5,
    topic="general",
)
structured_llm = llm.bind_tools([tavily_search_tool])
# agent = create_react_agent(llm, [tavily_search_tool])

# user_input = "What nation hosted the Euro 2024?"
user_input = "current price of 5090"
# formatted_prompt = web_searcher_instructions.format(
#     current_date=get_current_date(),
#     research_topic='price of 5090',
# )

web_searcher_instructions = """Use your search tool to conduct targeted Searches to gather the most recent, credible information on "{research_topic}" and synthesize it into a verifiable text artifact.

Instructions:
- Query should ensure that the most current information is gathered. The current date is {current_date}.
- Conduct multiple, diverse searches to gather comprehensive information.
- Consolidate key findings while meticulously tracking the source(s) for each specific piece of information.
- The output should be a well-written summary or report based on your search findings. 
- Only include the information found in the search results, don't make up any information.

Research Topic:
{research_topic}
"""

formatted_prompt = web_searcher_instructions.format(
    current_date=get_current_date(),
    research_topic=user_input,
) + "\n\nIMPORTANT: Use the search tool provided to find the most up-to-date information, and choose the right arguments for the search tool."



def run_tool(response_msg):
    if "tool_calls" in response_msg.additional_kwargs:
        for tool_call in response_msg.additional_kwargs["tool_calls"]:
            tool_name = tool_call["function"]["name"]
            arguments = tool_call["function"]["arguments"]
            parsed_args = json.loads(arguments)
            if tool_name == "tavily_search":
                return {"tool_result": tavily_search_tool.invoke(parsed_args)}
    return {"tool_result": "No tool call made."}

# === Final LLM with structured output ===
final_llm = llm.with_structured_output(QuotedAnswer)
# Step 3: Combine it with the LLM output step
chain = (
    tool_calling_llm |
    RunnableLambda(run_tool) |
    (lambda inputs: final_llm.invoke(f"Based on this search result: {inputs['tool_result']}, answer the user question."))
)

result = chain.invoke(formatted_prompt)
web_research_result = result.answer + "\n---\n\n".join([citation.source_id for citation in result.citations])
citations_list = []
for citation in result.citations:
    citations_dict = {}
    citations_dict['label'] = citation.label
    citations_dict['short_url'] = citation.source_id
    citations_dict['value'] = citation.quote
    citations_list.append(citations_dict)

# print(result)
# https://github.com/tavily-ai/use-cases/blob/main/company-research/company_research.ipynb

In [51]:

citations_list

[{'label': 'Wccftech',
  'short_url': 'https://wccftech.com/geforce-rtx-5090-drops-below-2800-for-the-first-time-in-japan/',
  'value': 'In EU regions, we can see most RTX 5090 editions dropping into the €2,400-€2,500 range in June, which was over €2,600 in May. It is still far from the MSRP of $1999, but hopefully, we can see sub-$2500 prices in the coming weeks.'},
 {'label': 'Wccftech',
  'short_url': 'https://wccftech.com/geforce-rtx-5090-drops-below-2800-for-the-first-time-in-japan/',
  'value': 'Both the GPUs are currently listed for 398,000 Yen or US$2775, which is a roughly $275 price reduction.'},
 {'label': 'Medium',
  'short_url': 'https://medium.com/@rosgluk/nvidia-rtx-5080-and-rtx-5090-prices-in-australia-june-2025-70fba640c134',
  'value': 'Mwave has Palit GeForce RTX 5090 GameRock for $5299 and ASUS TUF Gaming GeForce RTX 5090 for $5599.'}]

In [157]:
result.answer

"The current price of the NVIDIA GeForce RTX 5090, as of June 16, 2025, is not yet officially announced by NVIDIA. However, industry rumors and leaks suggest a potential price range of $1,800 to $2,000 based on speculation from tech analysts. For the most accurate and up-to-date information, it is recommended to check NVIDIA's official website or trusted tech news outlets."

In [148]:
# 'search_query': ['Euro 2024 winner'],
#  'web_research_result': ["Based on the search results, here's a summary of the Euro 2024 winner:\n\n**Spain won Euro 2024**, defeating England 2-1 in the final held in Berlin on July 14, 2024. [olympics](https://vertexaisearch.cloud.google.com/id/0-0) [youtube](https://vertexaisearch.cloud.google.com/id/0-1) [uefa](https://vertexaisearch.cloud.google.com/id/0-2) [uefa](https://vertexaisearch.cloud.google.com/id/0-3) This victory marks Spain's record-breaking fourth European Championship title. [uefa](https://vertexaisearch.cloud.google.com/id/0-3) [aljazeera](https://vertexaisearch.cloud.google.com/id/0-4) [youtube](https://vertexaisearch.cloud.google.com/id/0-1) [youtube](https://vertexaisearch.cloud.google.com/id/0-5) [wikipedia](https://vertexaisearch.cloud.google.com/id/0-6) [wikipedia](https://vertexaisearch.cloud.google.com/id/0-7) Spain won all seven of their matches during the tournament. [youtube](https://vertexaisearch.cloud.google.com/id/0-1) [wikipedia](https://vertexaisearch.cloud.google.com/id/0-7) [aljazeera](https://vertexaisearch.cloud.google.com/id/0-4) England has now lost back-to-back Euro finals. [aljazeera](https://vertexaisearch.cloud.google.com/id/0-4) [youtube](https://vertexaisearch.cloud.google.com/id/0-1) [wikipedia](https://vertexaisearch.cloud.google.com/id/0-7) The final score was 2-1. [olympics](https://vertexaisearch.cloud.google.com/id/0-0) [youtube](https://vertexaisearch.cloud.google.com/id/0-1) [uefa](https://vertexaisearch.cloud.google.com/id/0-2) [wikipedia](https://vertexaisearch.cloud.google.com/id/0-6) [soccerway](https://vertexaisearch.cloud.google.com/id/0-8) [uefa](https://vertexaisearch.cloud.google.com/id/0-3) [aljazeera](https://vertexaisearch.cloud.google.com/id/0-4) [wikipedia](https://vertexaisearch.cloud.google.com/id/0-7) Nico Williams and Mikel Oyarzabal scored for Spain in the final. [youtube](https://vertexaisearch.cloud.google.com/id/0-1)\n"],
#  'sources_gathered'

In [33]:
from langchain_tavily import TavilySearch
from langgraph.prebuilt import create_react_agent
from langchain_deepseek import ChatDeepSeek
from dotenv import load_dotenv
from backend.src.agent.prompts import (
    get_current_date,
    query_writer_instructions,
    web_searcher_instructions,
    reflection_instructions,
    answer_instructions,
)

load_dotenv()
llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=1,
    timeout=None,
    max_retries=2,
    )
# Create a ReAct agent with the web searcher system prompt and Tavily search tool
formatted_prompt = web_searcher_instructions.format(
    current_date=get_current_date(),
    research_topic='price of 5090',
)
tavily_search_tool = TavilySearch(
    max_results=5,
    topic="general",
)

# Create the agent with the formatted prompt as system message
agent = create_react_agent(
    llm,
    [tavily_search_tool],
    
)
prompt=formatted_prompt
# Create a runnable chain from the agent
# search_chain = agent.get_chain()

# agent.invoke({"messages": [HumanMessage(content="What is the price of 5090?")]}).pretty_print()

# user_input = "What is the price of 5090?"
# response = agent.invoke({"messages": prompt})
for step in agent.stream(
    {"messages": prompt},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()


================================ Human Message =================================

Conduct targeted Google Searches to gather the most recent, credible information on "price of 5090" and synthesize it into a verifiable text artifact.

Instructions:
- Query should ensure that the most current information is gathered. The current date is June 20, 2025.
- Conduct multiple, diverse searches to gather comprehensive information.
- Consolidate key findings while meticulously tracking the source(s) for each specific piece of information.
- The output should be a well-written summary or report based on your search findings. 
- Only include the information found in the search results, don't make up any information.

Research Topic:
price of 5090

================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_0_8040b8c7-8bb9-40ed-ad76-bb933e9bc2c5)
 Call ID: call_0_8040b8c7-8bb9-40ed-ad76-bb933e9bc2c5
  Args:
    query: price of NVIDIA RTX 5090 J

In [ ]:
query = "current price of RTX 5090 graphics card in Singapore dollars"
out = agent_exec.invoke(
        {
            "research_topic": query,
            "current_date": "20 June 2025",
            "input": query            # required by ReAct template
        }
    )

    # out["output"] is the raw JSON string from the LLM
    # parse it into the Pydantic model for convenience
qa = QuotedAnswer.model_validate_json(out["output"])
print("\n==== Final structured answer ====")
print(qa.model_dump_json(indent=2))
print("\n==== Intermediate steps ====")
print(out["intermediate_steps"])  

In [34]:
step

{'messages': [HumanMessage(content='Conduct targeted Google Searches to gather the most recent, credible information on "price of 5090" and synthesize it into a verifiable text artifact.\n\nInstructions:\n- Query should ensure that the most current information is gathered. The current date is June 20, 2025.\n- Conduct multiple, diverse searches to gather comprehensive information.\n- Consolidate key findings while meticulously tracking the source(s) for each specific piece of information.\n- The output should be a well-written summary or report based on your search findings. \n- Only include the information found in the search results, don\'t make up any information.\n\nResearch Topic:\nprice of 5090\n', additional_kwargs={}, response_metadata={}, id='c39caa5d-a57b-4679-80ef-3f95c70f821f'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0_8040b8c7-8bb9-40ed-ad76-bb933e9bc2c5', 'function': {'arguments': '{"query": "price of NVIDIA RTX 5090 June 2025", "search_dep

In [90]:
from IPython.display import Markdown

Markdown(response['messages'][4].content)

{"query": "latest updates on RTX 5090 pricing June 2025", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://x.com/USRestocks/status/1932361271782863013", "title": "US Restocks RTX5090/60/60Ti § Switch2", "content": "RTX 5090 in Stock! | Cart Link: https://amzn.to/4bPc14X June 10, 2025 at 01:56AM PT #ad MSI Gaming RTX 5090 32G Gaming Trio OC Graphics Card (32GB GDDR7,", "score": 0.79254496, "raw_content": null}, {"url": "https://www.youtube.com/watch?v=l641VkblkfY", "title": "Best GPUs to Buy for Gaming RIGHT NOW! [June/July 2025]", "content": "Newegg (US) [PAID LINKS]:\nB570: https://newegg.io/nc0f72caf3\nB580: https://newegg.io/ncb8aa2818\nRTX 5060: https://newegg.io/nc2d2d6cb4\nRTX 5060 Ti 16GB: https://newegg.io/nce151063f\nRX 9070: https://newegg.io/nc6c904af2\nRTX 5070 Ti: https://newegg.io/ncc8d843aa\nRX 9070 XT: https://newegg.io/ncd7924039\nRTX 5080: https://newegg.io/nc45487d71\nRTX 5090: https://newegg.io/nc60c8042e [...] Amazon (UK) [PAID LINKS]:\nB570: https://amzlink.to/az0zJQpeqq2Js\nB580: https://amzlink.to/az0LAlIxFBfKo\nRTX 5060: https://amzlink.to/az0hIVbEZrI6B\nRTX 5060 Ti 16GB: https://amzlink.to/az0HkaR5MIpvR\nRX 9070: https://amzlink.to/az0syS72HTuPw\nRTX 5070 Ti: https://amzlink.to/az0pZ6PXayEAU\nRX 9070 XT: https://amzlink.to/az0LwqrQy9bK1\nRTX 5080: https://amzlink.to/az0n4hQDRq9rx\nRTX 5090: https://amzlink.to/az0HXuNlzKZWX [...] Amazon (US) [PAID LINKS]:\nB570: https://amzlink.to/az0HOxxNV4OF8\nB580: https://amzlink.to/az0u4cH6r0fB2\nRTX 5060: https://amzlink.to/az0zbM77NOV2Z\nRTX 5060 Ti 16GB: https://amzlink.to/az0YZkOnMFXY3\nRX 9070: https://amzlink.to/az03UwkWLMGmH\nRTX 5070 Ti: https://amzlink.to/az0AItZVPbPNO\nRX 9070 XT: https://amzlink.to/az0oOI9UxwEay\nRTX 5080: https://amzlink.to/az0SSNK8AQuzo\nRTX 5090: https://amzlink.to/az0aj8ecrctEC", "score": 0.73023266, "raw_content": null}, {"url": "https://en.gamegpu.com/iron/msi-and-gigabyte-are-delaying-rtx-5090-deliveries-to-susa-until-power-rates-are-up-by-july-9", "title": "MSI and Gigabyte RTX 5090 Shipments to US Ahead of Tariffs ...", "content": "# MSI and Gigabyte RTX 5090 Shipments to US Ahead of Tariffs Coming into Effect on July 9\n\nAuthor [Maximum Games](https://en.gamegpu.com/view-user-profile?user=41287), 10 2025 June. Published in [Hardware](/backgammon/news/iron/) [...] ![rtx 5090 gigabyte](https://gamegpu.com/images/1_2025/NEWS/q2/f123/9600/rtx_5090_gigabyte.jpg)\n\n![rtx 5090 gigabyte](https://gamegpu.com/images/1_2025/NEWS/q2/f123/9600/rtx_5090_gigabyte.jpg) [...] Experts suggest that **NVIDIA's AIB Partners Will Hold Back RTX 5090 Stock**, waiting for the official return of tariffs. This will give the opportunity to raise prices after the deadline if agreements between the countries are not reached. A similar scheme was used in 2016-2017, when companies massively imported video cards before the start of trade restrictions. **RTX 5090 Availability May Worsen in July**, especially if tariffs are reintroduced and companies decide to pass the costs on to", "score": 0.65173227, "raw_content": null}, {"url": "https://www.notebookcheck.net/2025-Lenovo-Legion-Pro-7i-with-RTX-5090-and-Core-Ultra-9-275HX-gets-another-price-cut.1033540.0.html", "title": "2025 Lenovo Legion Pro 7i with RTX 5090 and Core Ultra 9 275HX ...", "content": "Deal | 2025 Lenovo Legion Pro 7i with RTX 5090 and Core Ultra 9 275HX gets another price cut ... It goes without saying that the top-end RTX 5090 configuration of", "score": 0.64719695, "raw_content": null}, {"url": "https://www.tomshardware.com/pc-components/gpus/cheapest-5090-ever-lucky-pc-builder-finds-rtx-5090-for-just-usd1-679-at-walmart-saving-usd1-520-card-has-worked-flawlessly-ever-since", "title": "Cheapest 5090 ever? Lucky PC builder finds RTX 5090 for 'just ...", "content": "Redditor Sufficient\\_Crazy7758 took to the platform to hail their purchase of the \"Cheapest 5090 ever?\", after scooping a PNY GeForce RTX 5090 from Walmart for the relatively low price of $1,679.99. The attached images reveal that the card was discounted by over $1,500 on May 25, having previously been discounted on May 13.\n\n\"Just so all of you know, I bought this on that date and it has worked flawlessly ever since,\" they said triumphantly. [...] In the ever-evolving world of insane GPU prices, tariffs, and frightful stock shortages, one intrepid PC builder has managed to swipe an Nvidia RTX 5090 from Walmart for \"just\" $1,679, more than $200 below MSRP. [...] even the [Founders Edition still retails for the same price](https://goto.walmart.com/c/1943169/565706/9383?subId1=tomshardware-us-9572975325724004555&sharedId=tomshardware-us&u=https%3A%2F%2Fwww.walmart.com%2Fip%2FNVIDIA-GeForce-RTX-5090-Graphic-Card-32GB-GDDR7%2F15176552726%3FclassType%3DREGULAR%26from%3D%2Fsearch), way over the $1,999 MSRP.", "score": 0.6016175, "raw_content": null}], "response_time": 2.98}

In [47]:
user_input = "What is the price of 5090?"

for step in agent.stream(
    {"messages": user_input},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is the price of 5090?
================================== Ai Message ==================================


In [52]:
import os

from backend.src.agent.tools_and_schemas import SearchQueryList, Reflection
from dotenv import load_dotenv
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.pydantic_v1 import BaseModel, Field
from langgraph.types import Send
from langgraph.graph import StateGraph
from langgraph.graph import START, END
from langchain_core.runnables import RunnableConfig
from google.genai import Client

from backend.src.agent.state import (
    OverallState,
    QueryGenerationState,
    ReflectionState,
    WebSearchState,
)
from backend.src.agent.configuration import Configuration
from backend.src.agent.prompts import (
    get_current_date,
    query_writer_instructions,
    web_searcher_instructions,
    reflection_instructions,
    answer_instructions,
)
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_deepseek import ChatDeepSeek

from backend.src.agent.utils import (
    get_citations,
    get_research_topic,
    insert_citation_markers,
    resolve_urls,
)
from typing import TypedDict, List, Annotated, Literal, Dict, Union, Optional 
from pydantic import BaseModel, Field
from langchain_tavily import TavilySearch
from langchain_core.runnables import RunnableLambda, RunnableSequence
import json

In [2]:
query_writer_instructions

'Your goal is to generate sophisticated and diverse web search queries. These queries are intended for an advanced automated web research tool capable of analyzing complex results, following links, and synthesizing information.\n\nInstructions: \n- think what you need to know to answer the question first, your query can be incremental, the firsr query answer can be used to generate the second query.\n- Always prefer a single search query, only add another query if the original question requests multiple aspects or elements and one query is not enough.\n- Each query should focus on one specific aspect of the original question.\n- Don\'t produce more than {number_queries} queries.\n- Queries should be diverse, if the topic is broad, generate more than 1 query.\n- Don\'t generate multiple similar queries, 1 is enough.\n- Query should ensure that the most current information is gathered. The current date is {current_date}.\n\nFormat: \n- Format your response as a JSON object with ALL three

In [53]:

def generate_query(state: OverallState, config: RunnableConfig)->QueryGenerationState :
    """LangGraph node that generates a search queries based on the User's question.

    Uses Gemini 2.0 Flash to create an optimized search query for web research based on
    the User's question.

    Args:
        state: Current graph state containing the User's question
        config: Configuration for the runnable, including LLM provider settings

    Returns:
        Dictionary with state update, including search_query key containing the generated query
    """
    configurable = Configuration.from_runnable_config(config)

    # check for custom initial search query count
    if state.get("initial_search_query_count") is None:
        state["initial_search_query_count"] = configurable.number_of_initial_queries

    # init Gemini 2.0 Flash
    # llm = ChatGoogleGenerativeAI(
    #     model=configurable.query_generator_model,
    #     temperature=1.0,
    #     max_retries=2,
    #     api_key=os.getenv("GEMINI_API_KEY"),
    # )
    llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=1,
    timeout=None,
    max_retries=2,
    )
    structured_llm = llm.with_structured_output(SearchQueryList)
    print(get_research_topic(state["messages"]),)
    # Format the prompt
    current_date = get_current_date()
    formatted_prompt = query_writer_instructions.format(
        current_date=current_date,
        research_topic=get_research_topic(state["messages"]),
        number_queries=state["initial_search_query_count"],
    )
    # Generate the search queries
    result = structured_llm.invoke(formatted_prompt)
    # state['plan'] = result.plan
    print(result)
    return {"plan": result.plan, "query_list": result.query}

builder = StateGraph(OverallState)

# Define the nodes we will cycle between
builder.add_node("generate_query", generate_query)

builder.add_edge(START, "generate_query")
# builder.add_conditional_edges(
#     "generate_query", continue_to_web_research, ["web_research"]
# )
builder.add_edge("generate_query", END)

graph = builder.compile(name="pro-search-agent")

In [22]:

state = graph.invoke({"messages": [{"role": "user", "content": "current price of 5090 with different brand,\
I want to buy a new gpu, and I am considering of Heat dissipation function and price, please recommend one to me"}], "max_research_loops": 5, "initial_search_query_count": 5})

current price of 5090 with different brand,I want to buy a new gpu, and I am considering of Heat dissipation function and price, please recommend one to me
query=['RTX 5090 GPU models comparison heat dissipation and price June 2025'] rationale='To recommend the best GPU based on heat dissipation and price, we need to gather current information on the latest RTX 5090 models from different brands, focusing on their cooling solutions and pricing. This will allow for a comparison of performance and value.' plan='1. Identify the latest RTX 5090 models available from different brands. 2. Compare their heat dissipation technologies and cooling solutions. 3. Analyze their current pricing to determine the best value for money. 4. Recommend the GPU that offers the best balance of cooling efficiency and affordability.'


In [23]:
# content='{\n    "rationale": "To recommend a GPU based on heat dissipation and price for the RTX 5090 models, two distinct data points are required: comprehensive thermal performance comparisons of various brands to evaluate cooling efficiency, and current market prices to assess value. These queries ensure coverage of technical reviews and real-time pricing for informed decision-making.",\n    "query": [\n        "RTX 5090 custom models thermal performance comparison 2025",\n        "RTX 5090 prices by brand June 2025"\n    ]\n}' additional_kwargs={'refusal': None, 'reasoning_content': 'We are given: current price of 5090 with different brand, and the user wants to buy a new GPU considering heat dissipation function and price.\n The user is specifically mentioning the "5090", which likely refers to the NVIDIA RTX 5090, a high-end graphics card. However, note that as of June 18, 2025, the RTX 5090 may or may not have been released (since NVIDIA\'s next-gen after RTX 40 series is expected around 2024-2025). We need to check the latest information.\n\n The user\'s requirements:\n   - Heat dissipation function (cooling performance)\n   - Price\n\n We are to recommend one GPU (presumably among the RTX 5090 models from different brands) that balances these two factors.\n\n Steps:\n 1. We need to gather current models of RTX 5090 from various brands (like ASUS, MSI, Gigabyte, Zotac, etc.) along with their cooling solutions and prices.\n 2. We need to compare these models based on:\n    - Cooling performance (which can be assessed through reviews that measure temperatures and noise levels under load)\n    - Price (the user has provided a current price context, but we need up-to-date pricing and deals)\n\n However, note: the user says "current price of 5090 with different brand", meaning they are aware of the existence of different brands and their varying prices. They want a recommendation based on heat dissipation and price.\n\n Approach for queries:\n Since we are to generate one or a few queries (max 5) and they should be specific and not redundant:\n\n Option 1: We can do one query that aims to get a comparison of RTX 5090 models from different brands, focusing on cooling and price.\n\n But note: the user might also be open to other GPUs? The question says "I want to buy a new GPU" and they are considering the 5090, but they might be open to other high-end models? However, the context specifically says "current price of 5090 with different brand", so it\'s clear they are focusing on the RTX 5090.\n\n However, let\'s stick to the context: they are considering the RTX 5090 from different brands.\n\n How about:\n\n Query 1: "RTX 5090 models cooling performance comparison 2025"\n   - This would help in getting reviews that compare the thermal performance of different custom models.\n\n Query 2: "RTX 5090 prices by brand June 2025"\n   - This would get the current prices.\n\n But wait: the user wants a recommendation. We are to generate queries that will help an automated tool to gather the necessary information to make that recommendation.\n\n However, note the instruction: "think what you need to know to answer the question first". We need to know:\n   - The available RTX 5090 models and their cooling solutions (and how effective they are)\n   - The current prices of these models\n\n But we can also look for a source that already does a comparison that includes both. So:\n\n Alternative: One query that covers both aspects? It might be hard to get one page that has both up-to-date prices and a detailed cooling comparison. Therefore, two queries might be necessary.\n\n However, let\'s consider: the current date is June 18, 2025. We can also look for the latest reviews and then separately look for the latest prices.\n\n But note: the instruction says "prefer a single search query" unless the question requests multiple aspects. Here we have two aspects: heat dissipation and price. So two queries are acceptable.\n\n But also, we might get a review that includes both performance (including thermal) and the MSRP. However, actual prices in the market may vary and may be found on retailer sites. So:\n\n Query 1: For cooling performance: "RTX 5090 thermal performance comparison 2025"\n   - This should return review roundups that test multiple models.\n\n Query 2: For current prices: "RTX 5090 current prices June 2025"\n   - This should return the latest prices from retailers or price comparison sites.\n\n However, note that we are limited to 5 queries and we should only add if necessary. Two queries seem sufficient.\n\n But wait: the user might also be interested in the overall value (price to performance including cooling). We might need to combine the two. However, the tool can do that if we get the two sets of data.\n\n Also, note: the user might be open to other GPUs? The context says they are considering the 5090, but if we are to strictly follow, we are focusing on the 5090. However, the question does not explicitly rule out other models. But the context of "current price of 5090 with different brand" suggests that the user is comparing within the 5090 models.\n\n Therefore, we generate two queries:\n\n  1. To get the thermal performance data of various RTX 5090 models.\n  2. To get the current prices of these models.\n\n But note: we might also need to know the brand and model names to cross-reference. The first query should return models by brand (like ASUS ROG Strix, MSI Suprim, etc.). The second query should also have the same models.\n\n However, we can also do one comprehensive query: "RTX 5090 models comparison 2025: cooling and price". But that might not be as effective.\n\n Considering the above, I propose two queries:\n\n   Query 1: "RTX 5090 custom models thermal performance review roundup 2025"\n   Query 2: "RTX 5090 prices by model June 2025"\n\n But note: the tool is advanced and can follow links and synthesize. So with these two, it can gather the thermal data and the price data and then combine to recommend the model with the best cooling in a reasonable price range (or the best value for cooling).\n\n However, we might also consider: what if the user wants the best cooling regardless of price? Or the best value? The question says "considering heat dissipation function and price". So we need both.\n\n Therefore, two queries.\n\n But wait, the instruction says: "Don\'t generate multiple similar queries, 1 is enough." if one query can cover. However, these two are distinct. They are about two different aspects.\n\n However, let\'s see if we can do with one:\n\n   Query: "RTX 5090 models comparison 2025: cooling and current prices"\n\n This might work? But it\'s possible that a single page might not have both in-depth thermal analysis and the very latest prices (which change frequently). So two queries might be more effective.\n\n Considering the instructions, we are allowed to have more than one if the question has multiple aspects. This one does. So two queries.\n\n But note: we must not exceed 5. Two is safe.\n\n Rationale: We need to compare different brands of the RTX 5090 based on their cooling performance and current prices to recommend the best option that balances these two factors. The first query targets thermal performance reviews to evaluate cooling efficiency, while the second query gathers up-to-date pricing information for the same models.\n\n Therefore:\n\n   "rationale": "To recommend an RTX 5090 model that balances heat dissipation and price, we need two distinct data points: comprehensive reviews comparing the cooling performance of various custom models, and current market prices for these models. These queries target the two critical factors independently to allow cross-referencing and value assessment.",\n   "query": [\n        "RTX 5090 custom models thermal performance review roundup 2025",\n        "RTX 5090 current prices by model June 2025"\n   ]\n\n However, note: the user might also be interested in other factors? But the question specifically says "Heat dissipation function and price". So we focus on these.\n\n But wait: the user might also care about noise? Often, cooling performance is tied to noise. But the question says "heat dissipation", so we focus on thermal performance. However, if the reviews measure temperatures, that\'s the key.\n\n Also, note: the model names must be consistent between the two queries. The first query should return model names (like ASUS ROG Strix, MSI Suprim X, etc.) and the second query should use the same naming.\n\n We are generating two queries. We are not generating more because two aspects are covered.\n\n Let\'s write the JSON accordingly.'} response_metadata={'token_usage': {'completion_tokens': 1862, 'prompt_tokens': 398, 'total_tokens': 2260, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 1752, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 64}, 'prompt_cache_hit_tokens': 64, 'prompt_cache_miss_tokens': 334}, 'model_name': 'deepseek-reasoner', 'system_fingerprint': 'fp_393bca965e_prod0425fp8', 'finish_reason': 'stop', 'logprobs': None} id='run--474f9824-8478-4634-966c-e979126eb49c-0' usage_metadata={'input_tokens': 398, 'output_tokens': 1862, 'total_tokens': 2260, 'input_token_details': {'cache_read': 64}, 'output_token_details': {'reasoning': 1752}}


In [20]:
state

{'messages': [HumanMessage(content='current price of 5090 with different brand,I want to buy a new gpu, and I am considering of Heat dissipation function and price, please recommend one to me', additional_kwargs={}, response_metadata={}, id='6056e78c-5d95-4b32-969c-2f90137c0996')],
 'search_query': [],
 'plan': '1. Retrieve the latest pricing and cooling solutions for the NVIDIA RTX 5090 across different brands. 2. Compare the heat dissipation efficiency and price to identify the best option. 3. Recommend the GPU with the optimal balance of cooling performance and affordability.',
 'web_research_result': [],
 'sources_gathered': [],
 'initial_search_query_count': 5,
 'max_research_loops': 5}

In [61]:
def continue_to_web_research(state: QueryGenerationState):
    """LangGraph node that sends the search queries to the web research node.

    This is used to spawn n number of web research nodes, one for each search query.
    """
    return [
        Send("web_research", {"search_query": search_query, "id": int(idx)})
        for idx, search_query in enumerate(state["query_list"])
    ]


# def web_research(state: WebSearchState, config: RunnableConfig) -> OverallState:
#     """LangGraph node that performs web research using the native Google Search API tool.

#     Executes a web search using the native Google Search API tool in combination with Gemini 2.0 Flash.

#     Args:
#         state: Current graph state containing the search query and research loop count
#         config: Configura"tion for the runnable, including search API settings

#     Returns:
#         Dictionary with state update, including sources_gathered, research_loop_count, and web_research_results
#     """
#     # Configure
#     configurable = Configuration.from_runnable_config(config)
#     formatted_prompt = web_searcher_instructions.format(
#         current_date=get_current_date(),
#         research_topic=state["search_query"],
#     )
#     llm = ChatDeepSeek(
#     model="deepseek-chat",
#     temperature=1,
#     timeout=None,
#     max_retries=2,
#     )

#     # Create the agent with the formatted prompt as system message
#     # agent = create_react_agent(
#     #     llm,
#     #     [tavily_search_tool],
#     #     prompt=formatted_prompt
#     # )
#     # Uses the google genai client as the langchain client doesn't return grounding metadata
#     genai_client = Client(api_key=os.getenv("GEMINI_API_KEY"))

#     response = genai_client.models.generate_content(
#         model=configurable.query_generator_model,
#         contents=formatted_prompt,
#         config={
#             "tools": [{"google_search": {}}],
#             "temperature": 0,
#         },
#     )
#     print('search results:')

#     print(response)
    
#     # resolve the urls to short urls for saving tokens and time
#     resolved_urls = resolve_urls(
#         response.candidates[0].grounding_metadata.grounding_chunks, state["id"]
#     )
#     # Gets the citations and adds them to the generated text
#     citations = get_citations(response, resolved_urls)
#     modified_text = insert_citation_markers(response.text, citations)
#     sources_gathered = [item for citation in citations for item in citation["segments"]]



    
#     return {
#         "sources_gathered": sources_gathered,
#         "search_query": [state["search_query"]],
#         "web_research_result": [modified_text],
#     }

In [62]:
class Citation(BaseModel):
    label: str = Field(
        ...,
        description="The name of the website.",
    )
    source_id: str = Field(
        ...,
        description="The url of a SPECIFIC source which justifies the answer.",
    )
    quote: str = Field(
        ...,
        description="The VERBATIM quote from the specified source that justifies the answer.",
    )


class QuotedAnswer(BaseModel):
    """Answer the user question based only on the given sources, and return all the sources used."""
    answer: str = Field(
        ...,
        description="The answer to the user question, which is based only on the given sources. Include any relevant sources in the answer as markdown hyperlinks. For example: 'This is a sample text ([url website](url))'"
    )
    citations: List[Citation] = Field(
        ..., description="Citations from the given sources that justify the answer."
    )



def web_research(state: WebSearchState, config: RunnableConfig) -> OverallState:
    """LangGraph node that performs web research using the native Google Search API tool.

    Executes a web search using the native Google Search API tool in combination with Gemini 2.0 Flash.

    Args:
        state: Current graph state containing the search query and research loop count
        config: Configura"tion for the runnable, including search API settings

    Returns:
        Dictionary with state update, including sources_gathered, research_loop_count, and web_research_results
    """
    # Configure
    load_dotenv()
    llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=1,
    timeout=None,
    max_retries=2,
    )


    # Initialize Tavily Search Tool
    tavily_search_tool = TavilySearch(
        max_results=5,
        topic="general",
    )
    structured_llm = llm.bind_tools([tavily_search_tool]).with_structured_output(QuotedAnswer)
    # agent = create_react_agent(llm, [tavily_search_tool])

    # user_input = "What nation hosted the Euro 2024?"
    user_input = state["search_query"]
    # formatted_prompt = web_searcher_instructions.format(
    #     current_date=get_current_date(),
    #     research_topic='price of 5090',
    # )

    web_searcher_instructions = """Use your search tool to conduct targeted Searches to gather the most recent, credible information on "{research_topic}" and synthesize it into a verifiable text artifact.

    Instructions:
    - Query should ensure that the most current information is gathered. The current date is {current_date}.
    - Conduct multiple, diverse searches to gather comprehensive information.
    - Consolidate key findings while meticulously tracking the source(s) for each specific piece of information.
    - The output should be a well-written summary or report based on your search findings. 
    - Only include the information found in the search results, don't make up any information.

    Research Topic:
    {research_topic}
    """

    formatted_prompt = web_searcher_instructions.format(
        current_date=get_current_date(),
        research_topic=user_input,
    ) + "\n\nIMPORTANT: Use the search tool provided to find the most up-to-date information, and choose the right arguments for the search tool."


    def run_tool(response_msg):
        if "tool_calls" in response_msg.additional_kwargs:
            for tool_call in response_msg.additional_kwargs["tool_calls"]:
                tool_name = tool_call["function"]["name"]
                arguments = tool_call["function"]["arguments"]
                parsed_args = json.loads(arguments)
                if tool_name == "tavily_search":
                    return {"tool_result": tavily_search_tool.invoke(parsed_args)}
        return {"tool_result": "No tool call made."}

    # === Final LLM with structured output ===
    final_llm = llm.with_structured_output(QuotedAnswer)
    # Step 3: Combine it with the LLM output step
    chain = (
        tool_calling_llm |
        RunnableLambda(run_tool) |
        (lambda inputs: final_llm.invoke(f"Based on this search result: {inputs['tool_result']}, answer the user question."))
    )

    result = chain.invoke(formatted_prompt)
    web_research_result = result.answer + "\n---\n\n".join([citation.source_id for citation in result.citations])
    citations_list = []
    for citation in result.citations:
        citations_dict = {}
        citations_dict['label'] = citation.label
        citations_dict['short_url'] = citation.source_id
        citations_dict['value'] = citation.quote
        citations_list.append(citations_dict)

    
    return {
        "sources_gathered": citations_list,
        "search_query": [state["search_query"]],
        "web_research_result": [web_research_result],
    }

In [27]:
builder = StateGraph(OverallState)

# Define the nodes we will cycle between
builder.add_node("generate_query", generate_query)
builder.add_node("web_research", web_research)

builder.add_edge(START, "generate_query")
builder.add_conditional_edges(
    "generate_query", continue_to_web_research, ["web_research"]
)
builder.add_edge("web_research", END)

graph = builder.compile(name="pro-search-agent")

In [28]:

state = graph.invoke({"messages": [{"role": "user", "content": "current price of 5090 with different brand,\
I want to buy a new gpu, and I am considering Heat dissipation function and price, please recommend one to me"}], "max_research_loops": 5, "initial_search_query_count": 5})

current price of 5090 with different brand,I want to buy a new gpu, and I am considering Heat dissipation function and price, please recommend one to me
query=['NVIDIA RTX 5090 price comparison June 2025', 'NVIDIA RTX 5090 heat dissipation performance reviews June 2025'] rationale='To recommend the best GPU based on price and heat dissipation, we need current pricing data for the NVIDIA RTX 5090 across different brands and reviews evaluating its heat dissipation performance. These queries will provide the necessary information to compare options.' plan='1. Retrieve the latest pricing for the NVIDIA RTX 5090 from different brands to identify cost-effective options. 2. Gather reviews or benchmarks on heat dissipation performance to ensure the GPU meets cooling requirements. 3. Synthesize the data to recommend the best option balancing price and heat dissipation.'


In [29]:
state 

{'messages': [HumanMessage(content='current price of 5090 with different brand,I want to buy a new gpu, and I am considering Heat dissipation function and price, please recommend one to me', additional_kwargs={}, response_metadata={}, id='2ea6b91f-93c8-43e4-a46f-b3b407c59c7b')],
 'search_query': ['NVIDIA RTX 5090 price comparison June 2025',
  'NVIDIA RTX 5090 heat dissipation performance reviews June 2025'],
 'plan': '1. Retrieve the latest pricing for the NVIDIA RTX 5090 from different brands to identify cost-effective options. 2. Gather reviews or benchmarks on heat dissipation performance to ensure the GPU meets cooling requirements. 3. Synthesize the data to recommend the best option balancing price and heat dissipation.',
 'web_research_result': ['As of June 2025, the NVIDIA RTX 5090 is priced at approximately $1,999 USD for the Founders Edition, with third-party variants ranging from $2,099 to $2,499 depending on the manufacturer and additional features like overclocking or enha

In [63]:
def reflection(state: OverallState, config: RunnableConfig) -> ReflectionState:
    """LangGraph node that identifies knowledge gaps and generates potential follow-up queries.

    Analyzes the current summary to identify areas for further research and generates
    potential follow-up queries. Uses structured output to extract
    the follow-up query in JSON format.

    Args:
        state: Current graph state containing the running summary and research topic
        config: Configuration for the runnable, including LLM provider settings

    Returns:
        Dictionary with state update, including search_query key containing the generated follow-up query
    """
    # configurable = Configuration.from_runnable_config(config)
    # Increment the research loop count and get the reasoning model
    state["research_loop_count"] = state.get("research_loop_count", 0) + 1
    # reasoning_model = state.get("reasoning_model") or configurable.reasoning_model

    # Format the prompt
    current_date = get_current_date()
    formatted_prompt = reflection_instructions.format(
        current_date=current_date,
        research_topic=get_research_topic(state["messages"]),
        summaries="\n\n---\n\n".join(state["web_research_result"]),
        plan=state["plan"],
    )
    # init Reasoning Model
    # llm = ChatGoogleGenerativeAI(
    #     model=reasoning_model,
    #     temperature=1.0,
    #     max_retries=2,
    #     api_key=os.getenv("GEMINI_API_KEY"),
    # )
    llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=1,
    max_tokens=1024,
    timeout=None,
    max_retries=2,
    )
    result = llm.with_structured_output(Reflection).invoke(formatted_prompt)

    return {
        "is_sufficient": result.is_sufficient,
        "knowledge_gap": result.knowledge_gap,
        "follow_up_queries": result.follow_up_queries,
        "research_loop_count": state["research_loop_count"],
        "number_of_ran_queries": len(state["search_query"]),
    }


def evaluate_research(
    state: ReflectionState,
    config: RunnableConfig,
) -> OverallState:
    """LangGraph routing function that determines the next step in the research flow.

    Controls the research loop by deciding whether to continue gathering information
    or to finalize the summary based on the configured maximum number of research loops.

    Args:
        state: Current graph state containing the research loop count
        config: Configuration for the runnable, including max_research_loops setting

    Returns:
        String literal indicating the next node to visit ("web_research" or "finalize_summary")
    """
    configurable = Configuration.from_runnable_config(config)
    max_research_loops = (
        state.get("max_research_loops")
        if state.get("max_research_loops") is not None
        else configurable.max_research_loops
    )
    if state["is_sufficient"] or state["research_loop_count"] >= max_research_loops:
        return "finalize_answer"
    else:
        return [
            Send(
                "web_research",
                {
                    "search_query": follow_up_query,
                    "id": state["number_of_ran_queries"] + int(idx),
                },
            )
            for idx, follow_up_query in enumerate(state["follow_up_queries"])
        ]


def finalize_answer(state: OverallState, config: RunnableConfig):
    """LangGraph node that finalizes the research summary.

    Prepares the final output by deduplicating and formatting sources, then
    combining them with the running summary to create a well-structured
    research report with proper citations.

    Args:
        state: Current graph state containing the running summary and sources gathered

    Returns:
        Dictionary with state update, including running_summary key containing the formatted final summary with sources
    """
    configurable = Configuration.from_runnable_config(config)
    # reasoning_model = state.get("reasoning_model") or configurable.reasoning_model

    # Format the prompt
    current_date = get_current_date()
    formatted_prompt = answer_instructions.format(
        current_date=current_date,
        research_topic=get_research_topic(state["messages"]),
        summaries="\n---\n\n".join(state["web_research_result"]),
    )

    # init Reasoning Model, default to Gemini 2.5 Flash
    # llm = ChatGoogleGenerativeAI(
    #     model=reasoning_model,
    #     temperature=0,
    #     max_retries=2,
    #     api_key=os.getenv("GEMINI_API_KEY"),
    # )
    llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=1,
    max_tokens=1024,
    timeout=None,
    max_retries=2,
    )
    result = llm.invoke(formatted_prompt)

    # Replace the short urls with the original urls and add all used urls to the sources_gathered
    unique_sources = []
    for source in state["sources_gathered"]:
        if source["short_url"] in result.content:
            result.content = result.content.replace(
                source["short_url"], source["value"]
            )
            unique_sources.append(source)

    return {
        "messages": [AIMessage(content=result.content)],
        "sources_gathered": unique_sources,
    }


# Create our Agent Graph
builder = StateGraph(OverallState, config_schema=Configuration)

# Define the nodes we will cycle between
builder.add_node("generate_query", generate_query)
builder.add_node("web_research", web_research)
builder.add_node("reflection", reflection)
builder.add_node("finalize_answer", finalize_answer)

# Set the entrypoint as `generate_query`
# This means that this node is the first one called
builder.add_edge(START, "generate_query")
# Add conditional edge to continue with search queries in a parallel branch
builder.add_conditional_edges(
    "generate_query", continue_to_web_research, ["web_research"]
)
# Reflect on the web research
builder.add_edge("web_research", "reflection")
# Evaluate the research
builder.add_conditional_edges(
    "reflection", evaluate_research, ["web_research", "finalize_answer"]
)
# Finalize the answer
builder.add_edge("finalize_answer", END)

graph = builder.compile(name="pro-search-agent")

In [6]:

state = graph.invoke({"messages": [{"role": "user", "content": "current price of 5090 with different brand,\
I want to buy a new gpu, and I am considering Heat dissipation function and price, please recommend one to me"}], "max_research_loops": 5, "initial_search_query_count": 5})

current price of 5090 with different brand,I want to buy a new gpu, and I am considering Heat dissipation function and price, please recommend one to me
query=['RTX 5090 GPU models price comparison June 2025', 'RTX 5090 GPU thermal performance and cooling solutions June 2025'] rationale='To recommend the best GPU based on heat dissipation and price, we need to gather current pricing and thermal performance data for the RTX 5090 models from different brands. This will allow for a comparison of cost-effectiveness and cooling efficiency.' plan='1. Retrieve the latest pricing information for RTX 5090 GPUs from various brands to identify cost-effective options. 2. Gather data on thermal performance and cooling solutions for these models to evaluate heat dissipation efficiency. 3. Compare the collected data to recommend the best GPU based on the balance between price and cooling performance.'


In [64]:

state = graph.invoke({"messages": [{"role": "user", "content": "If I can buy a switch 2 in RMB 3300. Do you think it's worth?\
    I am hesitatig, because there are not too many new games for switch2.  "}], "max_research_loops": 5, "initial_search_query_count": 5})

If I can buy a switch 2 in RMB 3300. Do you think it's worth?    I am hesitatig, because there are not too many new games for switch2.  
query=['Nintendo Switch 2 release date and price in RMB', 'Nintendo Switch 2 game lineup 2025'] rationale='To determine if the Switch 2 is worth buying for RMB 3300, we need to verify its official price in RMB and assess the current and upcoming game lineup. This will help evaluate whether the cost aligns with the value provided by available games.' plan='1. Confirm the official price of the Nintendo Switch 2 in RMB to compare with the offered price of RMB 3300. 2. Investigate the game lineup for the Switch 2 in 2025 to assess the availability of new and appealing games. 3. Combine the findings to make an informed decision on the purchase.'


In [65]:
state

{'messages': [HumanMessage(content="If I can buy a switch 2 in RMB 3300. Do you think it's worth?    I am hesitatig, because there are not too many new games for switch2.  ", additional_kwargs={}, response_metadata={}, id='c86843e6-3683-44e8-bddf-1e391999ff40'),
  AIMessage(content='Based on the available information, purchasing the Nintendo Switch 2 for **RMB 3300** (approximately **$460**) seems like a reasonable deal, especially considering that stolen units were valued at around **$500 (~RMB 3594)** ([Passionategeekz](Nintendo Switch 2 game consoles were stolen, each worth about US$500 (Passionategeekz Note: the current exchange rate is about RMB 3594).)). Here’s why it might be worth it:  \n\n### **1. Strong Launch and Upcoming Game Lineup**  \nThe Switch 2 launched on **June 5, 2025**, with several high-profile games already available or confirmed for later in the year:  \n- **Mario Kart World** – Topping sales charts and praised as "the perfect introduction to a new console gene

In [66]:
state['sources_gathered']

[{'label': 'Nintendo Life',
  'short_url': 'https://www.nintendolife.com/news/2025/05/rumour-a-nintendo-direct-may-drop-around-the-switch-2s-launch',
  'value': 'Meanwhile, the Switch 2 itself is launching worldwide on 5th June 2025, alongside launch games such as Mario Kart World, Cyberpunk 2077: Ultimate Edition, Street Fighter 6: Years 1-2 Fighters Edition, and more.'},
 {'label': 'Nintendo Life',
  'short_url': 'https://www.nintendolife.com/news/2025/05/pokemon-legends-z-a-secures-switch-and-switch-2-release-date',
  'value': 'The Pokémon Company has surprised us all with the announcement that Pokémon Legends: Z-A will arrive on Switch and Switch 2 on 16th October 2025.'},
 {'label': 'Nintendo Everything',
  'short_url': 'https://nintendoeverything.com/devils-third-interview-wii-u-tomonobu-itagaki/',
  'value': "Image 17: Famitsu's most wanted games - June 8, 2025 - Metroid Prime 4 - Nintendo Switch 2 Edition appears"},
 {'label': 'Nintendo Life',
  'short_url': 'https://www.ninten

In [67]:
from IPython.display import Markdown

Markdown(state['messages'][1].content)

Based on the available information, purchasing the Nintendo Switch 2 for **RMB 3300** (approximately **$460**) seems like a reasonable deal, especially considering that stolen units were valued at around **$500 (~RMB 3594)** ([Passionategeekz](Nintendo Switch 2 game consoles were stolen, each worth about US$500 (Passionategeekz Note: the current exchange rate is about RMB 3594).)). Here’s why it might be worth it:  

### **1. Strong Launch and Upcoming Game Lineup**  
The Switch 2 launched on **June 5, 2025**, with several high-profile games already available or confirmed for later in the year:  
- **Mario Kart World** – Topping sales charts and praised as "the perfect introduction to a new console generation" ([Nintendo Life](Image 92: Review: Mario Kart World (Switch 2) - The Perfect Introduction To A New Console Generation)).  
- **Cyberpunk 2077: Ultimate Edition** – A fully optimized version for the Switch 2 ([Nintendo Life](Review: Fast Fusion (Switch 2) - The Most Accomplish...)).  
- **Pokémon Legends: Z-A** – Releasing on **October 16, 2025**, with enhanced graphics and performance ([Nintendo Life](The Pokémon Company has surprised us all with the announcement that Pokémon Legends: Z-A will arrive on Switch and Switch 2 on 16th October 2025.)).  
- **Metroid Prime 4** – A highly anticipated exclusive ([Nintendo Everything](Image 17: Famitsu's most wanted games - June 8, 2025 - Metroid Prime 4 - Nintendo Switch 2 Edition appears)).  
- **Fast Fusion** & **Misc. A Tiny Tale** – Both praised for their smooth performance on the Switch 2 ([Nintendo Life]('Misc. A Tiny Tale' Looks Super Smooth On Switch 2, Thanks To A Dev Hunch)).  

Additionally, **The Legend of Zelda: Tears of the Kingdom** and **Echoes of Wisdom** have been enhanced for the Switch 2 ([TechRadar]("The Legend of Zelda: Tears of the Kingdom leaps forward on what was offered in 2017’s Breath of the Wild, filling out the world’s sandbox with innovative new tools and abilities, letting you approach challenges in inventive new ways." - Julian Benson, TechRadar Gaming reviewer.)).  

### **2. Backward Compatibility & Future-Proofing**  
The Switch 2 supports **both physical and digital Nintendo Switch games**, meaning your existing library will carry over ([ScreenRant](The Nintendo Switch 2 is the successor to Nintendo Switch, scheduled for release in 2025. Confirmed as backwards compatible, it will play both physical and digital Nintendo Switch games.), [TheGamer](The Nintendo Switch 2 is the successor to Nintendo Switch, scheduled for release on June 5, 2025. Confirmed as backwards compatible, it will play both physical and digital Nintendo Switch games.)). This makes it a safer investment, even if the initial game lineup isn’t massive.  

### **3. Price Justification**  
At **RMB 3300**, you're paying slightly below the estimated market value (~RMB 3594), making it a decent deal. If you're interested in upcoming exclusives like *Pokémon Legends: Z-A* or enhanced versions of existing games, the purchase is justified.  

### **Final Verdict**  
If you enjoy Nintendo exclusives or plan to play enhanced versions of Switch games, **RMB 3300 is a fair price**. However, if you're solely waiting for a larger library, you might want to hold off until late 2025 when more titles (like *Pokémon Legends: Z-A*) release.  

Would you like recommendations on specific games based on your preferences?

In [188]:
from IPython.display import Markdown

Markdown(state['messages'][1].content)

Here’s a comprehensive answer to your question about the current pricing of the RTX 5090 across different brands and regions as of June 2025:

### **RTX 5090 Pricing by Brand (June 2025)**  
1. **NVIDIA Founders Edition**: $1,599 ([TechRadar](The base model is priced at $1,599.))  
2. **ASUS ROG Strix**: $1,699 ([Tom's Hardware](The ASUS variant is listed at $1,699.))  
3. **MSI Gaming X Trio**: $1,649 ([PC Gamer](MSI's offering is priced at $1,649.))  
4. **Gigabyte AORUS**: $1,679 ([AnandTech](The Gigabyte model is available for $1,679.))  

### **Regional Pricing and Promotions**  
- **North America**:  
  - Founders Edition: $1,999  
  - ASUS/MSI models: $2,099–$2,299  
  - Newegg promotion: $100 discount on select models ([Newegg](Newegg is currently running a promotion with a $100 discount on select models.)).  

- **Europe**:  
  - Germany: Founders Edition at €2,199; MSI models with €150 discount ([Amazon Germany](Amazon Germany is offering a limited-time €150 discount on MSI models.)).  
  - France: Founders Edition at €2,249.  

- **Asia**:  
  - Japan: Founders Edition at ¥320,000; discounts up to ¥20,000 at Bic Camera ([Bic Camera](In Japan, the RTX 5090 is priced at ¥320,000 for the Founders Edition, with discounts of up to ¥20,000 on select models at Bic Camera.)).  
  - China: ¥14,999 with a ¥500 discount on JD.com ([JD.com](In China, the card is priced at ¥14,999, with JD.com offering a ¥500 discount for a limited time.)).  

- **Australia**:  
  - Founders Edition: AUD 3,299 with free game bundle at Scorptec ([Scorptec](Retailers like Scorptec offering a free game bundle with purchase.)).  

### **Performance Comparison**  
- **ASUS ROG Strix**: Highest performance (5–10% over Founders Edition) with triple-fan cooling ([Tom's Hardware](The ASUS variant is listed at $1,699.)).  
- **MSI Gaming X Trio**: Quiet operation, slightly outperforms Founders Edition ([Guru3D](It offers competitive performance, slightly edging out the Founders Edition in synthetic benchmarks but trailing the ROG Strix in gaming performance.)).  
- **Gigabyte AORUS**: Matches ASUS in clock speeds but louder under load ([AnandTech](The Gigabyte model is available for $1,679.)).  

Prices and promotions are subject to change. For the latest updates, check retailer links or official brand pages.  

**Citations**:  
[TechRadar](The base model is priced at $1,599.), [Tom's Hardware](The ASUS variant is listed at $1,699.), [PC Gamer](MSI's offering is priced at $1,649.), [AnandTech](The Gigabyte model is available for $1,679.), [Newegg](Newegg is currently running a promotion with a $100 discount on select models.), [Amazon Germany](Amazon Germany is offering a limited-time €150 discount on MSI models.), [Bic Camera](In Japan, the RTX 5090 is priced at ¥320,000 for the Founders Edition, with discounts of up to ¥20,000 on select models at Bic Camera.), [JD.com](In China, the card is priced at ¥14,999, with JD.com offering a ¥500 discount for a limited time.), [Scorptec](Retailers like Scorptec offering a free game bundle with purchase.), [Guru3D](It offers competitive performance, slightly edging out the Founders Edition in synthetic benchmarks but trailing the ROG Strix in gaming performance.).

In [ ]:
builder = StateGraph(OverallState, config_schema=Configuration)

# Define the nodes we will cycle between
builder.add_node("generate_query", generate_query)
builder.add_node("web_research", web_research)
builder.add_edge(START, "generate_query")
builder.add_conditional_edges(
    "generate_query", continue_to_web_research, ["web_research"]
)

builder.add_edge("finalize_answer", END)

graph = builder.compile(name="pro-search-agent")


In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.messages import AIMessage, HumanMessage

class LLMGeneratedSearchQueries(BaseModel):
    """Schema for the LLM to generate a list of search queries."""
    search_queries: list[str] = Field(description="A list of search queries to find relevant information based on the research topic.")

llm = ChatDeepSeek(
    model="deepseek-chat",
        temperature=0.0,
        max_tokens=1024,
        timeout=None,
        max_retries=2,
    )

messages = [HumanMessage(content=formatted_prompt)]

query_generation_response = llm.invoke(
    messages,
    tools=[LLMGeneratedSearchQueries]
)

generated_queries = []
if query_generation_response.tool_calls:
    for tool_call in query_generation_response.tool_calls:
        if tool_call['name'] == LLMGeneratedSearchQueries.__name__:
            generated_queries.extend(tool_call['args'].get('search_queries', []))

TypeError: argument of type 'ModelMetaclass' is not iterable